In [ ]:
##Q1: To Create SQLalchemy engine from the sample data

In [17]:
import pandas as pd
from pandas import DataFrame, Series
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine

In [13]:
##To read the data from the given url
x = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", header=None)
adult = pd.DataFrame(x, index=None)
adult = adult.rename(columns={0:'age', 1:'workclass', 2:'fnlwgt', 3:'education', 4:'educationnum', 5:'maritalstatus', 6:'occupation', 7:'relationship', 8:'race', 9:'gender', 10:'capital-gain', 11:'capitalloss', 12:'hoursperweek', 13:'nativecountry'})
adult.to_csv(r"C:\Users\navne\Documents\Python Scripts\Data\adult_data2.csv")

In [14]:
adult.head()

,age,workclass,fnlwgt,education,educationnum,maritalstatus,occupation,relationship,race,gender,capital-gain,capitalloss,hoursperweek,nativecountry,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [18]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [19]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

In [91]:
Base = declarative_base()
class Adult(Base):
    __tablename__ = 'Adult_Data'
    __table_args__ = {'sqlite_autoincrement': True}
    # To tell SQLAlchemy the name of column and its attributes:
    id = Column(Integer, primary_key=True, nullable=False) 
    age = Column(String)
    workclass = Column(Integer)
    education = Column(Integer)
    gender = Column(String)
    nativecountry = Column(String)
      
    def __repr__(self):
        return "<Adult(age='%s', workclass='%s', education='%s', gender='%s', nativecountry=%s)>" %(self.age, self.workclass, self.education, self.gender, self.nativecountry)


In [59]:
Adult.__table__

Table('Adult_Data', MetaData(bind=None), Column('id', Integer(), table=<Adult_Data>, primary_key=True, nullable=False), Column('age', String(), table=<Adult_Data>), Column('workclass', Integer(), table=<Adult_Data>), Column('education', Integer(), table=<Adult_Data>), Column('gender', String(), table=<Adult_Data>), Column('nativecountry', String(), table=<Adult_Data>), schema=None)

In [60]:
Base.metadata.create_all(engine)

2018-08-06 20:04:23,917 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Adult_Data")
2018-08-06 20:04:23,918 INFO sqlalchemy.engine.base.Engine ()


In [61]:
#To create the sample data 
fields = ['age', 'workclass', 'education', 'gender', 'nativecountry']
df = pd.read_csv(r"C:\Users\navne\Documents\Python Scripts\Data\adult_data2.csv", skipinitialspace=True, usecols=fields )
df.to_sql(con=engine, index_label='id', name=Adult.__tablename__, if_exists='replace')

2018-08-06 20:04:24,518 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Adult_Data")
2018-08-06 20:04:24,519 INFO sqlalchemy.engine.base.Engine ()
2018-08-06 20:04:24,521 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Adult_Data")
2018-08-06 20:04:24,521 INFO sqlalchemy.engine.base.Engine ()
2018-08-06 20:04:24,522 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-08-06 20:04:24,524 INFO sqlalchemy.engine.base.Engine ()
2018-08-06 20:04:24,526 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Adult_Data")
2018-08-06 20:04:24,526 INFO sqlalchemy.engine.base.Engine ()
2018-08-06 20:04:24,528 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Adult_Data' AND type = 'table'
2018-08-06 20:04:24,528 INFO sqlalchemy.engine.base.Engine ()
2018-08-06 20:04:24,530 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("Adult_Data")
2

In [62]:
from sqlalchemy.orm import sessionmaker

In [63]:
Session = sessionmaker(bind=engine)

In [64]:
Session.configure(bind=engine)

In [65]:
ssession = Session()

In [66]:
##Q2: Write two basic update queries

In [92]:
first_update = ssession.query(Adult).filter_by(age='17').first()

2018-08-06 20:17:13,591 INFO sqlalchemy.engine.base.Engine UPDATE "Adult_Data" SET education=? WHERE "Adult_Data".id = ?
2018-08-06 20:17:13,592 INFO sqlalchemy.engine.base.Engine ('Undergraduate', 106)
2018-08-06 20:17:13,594 INFO sqlalchemy.engine.base.Engine SELECT "Adult_Data".id AS "Adult_Data_id", "Adult_Data".age AS "Adult_Data_age", "Adult_Data".workclass AS "Adult_Data_workclass", "Adult_Data".education AS "Adult_Data_education", "Adult_Data".gender AS "Adult_Data_gender", "Adult_Data".nativecountry AS "Adult_Data_nativecountry" 
FROM "Adult_Data" 
WHERE "Adult_Data".age = ?
 LIMIT ? OFFSET ?
2018-08-06 20:17:13,595 INFO sqlalchemy.engine.base.Engine ('17', 1, 0)


In [93]:
first_update

<Adult(age='17', workclass='?', education='Undergraduate', gender='Female', nativecountry=United-States)>

In [98]:
first_update.workclass = 'Never-worked'   ##To update the workclass of the above entry from '?' 

In [99]:
first_update

<Adult(age='17', workclass='Never-worked', education='Undergraduate', gender='Female', nativecountry=United-States)>

In [96]:
update2 = session.query(Adult).filter_by(nativecountry='?').first()


2018-08-06 20:17:34,693 INFO sqlalchemy.engine.base.Engine SELECT "Adult_Data".id AS "Adult_Data_id", "Adult_Data".age AS "Adult_Data_age", "Adult_Data".workclass AS "Adult_Data_workclass", "Adult_Data".education AS "Adult_Data_education", "Adult_Data".gender AS "Adult_Data_gender", "Adult_Data".nativecountry AS "Adult_Data_nativecountry" 
FROM "Adult_Data" 
WHERE "Adult_Data".nativecountry = ?
 LIMIT ? OFFSET ?
2018-08-06 20:17:34,694 INFO sqlalchemy.engine.base.Engine ('?', 1, 0)


In [97]:
update2

<Adult(age='40', workclass='Private', education='Assoc-voc', gender='Male', nativecountry=?)>

In [100]:
update2.nativecountry = 'India'   ##To update the nativecountry of the above entry from '?' 

In [101]:
update2

<Adult(age='40', workclass='Private', education='Assoc-voc', gender='Male', nativecountry=India)>

In [ ]:
##Write two delete queries

In [102]:
delete_entry1 = session.query(Adult).first()

2018-08-06 20:21:55,260 INFO sqlalchemy.engine.base.Engine UPDATE "Adult_Data" SET nativecountry=? WHERE "Adult_Data".id = ?
2018-08-06 20:21:55,261 INFO sqlalchemy.engine.base.Engine ('India', 14)
2018-08-06 20:21:55,263 INFO sqlalchemy.engine.base.Engine SELECT "Adult_Data".id AS "Adult_Data_id", "Adult_Data".age AS "Adult_Data_age", "Adult_Data".workclass AS "Adult_Data_workclass", "Adult_Data".education AS "Adult_Data_education", "Adult_Data".gender AS "Adult_Data_gender", "Adult_Data".nativecountry AS "Adult_Data_nativecountry" 
FROM "Adult_Data"
 LIMIT ? OFFSET ?
2018-08-06 20:21:55,264 INFO sqlalchemy.engine.base.Engine (1, 0)


In [103]:
delete_entry1

<Adult(age='39', workclass='State-gov', education='Bachelors', gender='Male', nativecountry=United-States)>

In [104]:
session.delete(delete_entry1)
session.commit()

2018-08-06 20:22:38,324 INFO sqlalchemy.engine.base.Engine DELETE FROM "Adult_Data" WHERE "Adult_Data".id = ?
2018-08-06 20:22:38,325 INFO sqlalchemy.engine.base.Engine (0,)
2018-08-06 20:22:38,326 INFO sqlalchemy.engine.base.Engine COMMIT


In [106]:
delete_entry1

<Adult(age='39', workclass='State-gov', education='Bachelors', gender='Male', nativecountry=United-States)>

In [110]:
##To check whether the entry is deleted. If the output of the following has changed, the earlier entry is deleted
delete_entry1 = session.query(Adult).first()
delete_entry1

2018-08-06 20:24:08,670 INFO sqlalchemy.engine.base.Engine SELECT "Adult_Data".id AS "Adult_Data_id", "Adult_Data".age AS "Adult_Data_age", "Adult_Data".workclass AS "Adult_Data_workclass", "Adult_Data".education AS "Adult_Data_education", "Adult_Data".gender AS "Adult_Data_gender", "Adult_Data".nativecountry AS "Adult_Data_nativecountry" 
FROM "Adult_Data"
 LIMIT ? OFFSET ?
2018-08-06 20:24:08,671 INFO sqlalchemy.engine.base.Engine (1, 0)


<Adult(age='50', workclass='Self-emp-not-inc', education='Bachelors', gender='Male', nativecountry=United-States)>

In [111]:
delete_entry2 = session.query(Adult).filter_by(nativecountry='?').first()

2018-08-06 20:24:22,996 INFO sqlalchemy.engine.base.Engine SELECT "Adult_Data".id AS "Adult_Data_id", "Adult_Data".age AS "Adult_Data_age", "Adult_Data".workclass AS "Adult_Data_workclass", "Adult_Data".education AS "Adult_Data_education", "Adult_Data".gender AS "Adult_Data_gender", "Adult_Data".nativecountry AS "Adult_Data_nativecountry" 
FROM "Adult_Data" 
WHERE "Adult_Data".nativecountry = ?
 LIMIT ? OFFSET ?
2018-08-06 20:24:22,997 INFO sqlalchemy.engine.base.Engine ('?', 1, 0)


In [112]:
delete_entry2

<Adult(age='31', workclass='Private', education='Some-college', gender='Male', nativecountry=?)>

In [113]:
session.delete(delete_entry2)
session.commit()

2018-08-06 20:25:02,971 INFO sqlalchemy.engine.base.Engine DELETE FROM "Adult_Data" WHERE "Adult_Data".id = ?
2018-08-06 20:25:02,972 INFO sqlalchemy.engine.base.Engine (38,)
2018-08-06 20:25:02,972 INFO sqlalchemy.engine.base.Engine COMMIT


In [114]:
##To check whether the entry is deleted.If the output of the following has changed, the earlier entry is deleted
delete_entry2 = session.query(Adult).filter_by(nativecountry='?').first()
delete_entry2

2018-08-06 20:25:20,163 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-08-06 20:25:20,165 INFO sqlalchemy.engine.base.Engine SELECT "Adult_Data".id AS "Adult_Data_id", "Adult_Data".age AS "Adult_Data_age", "Adult_Data".workclass AS "Adult_Data_workclass", "Adult_Data".education AS "Adult_Data_education", "Adult_Data".gender AS "Adult_Data_gender", "Adult_Data".nativecountry AS "Adult_Data_nativecountry" 
FROM "Adult_Data" 
WHERE "Adult_Data".nativecountry = ?
 LIMIT ? OFFSET ?
2018-08-06 20:25:20,166 INFO sqlalchemy.engine.base.Engine ('?', 1, 0)


<Adult(age='18', workclass='Private', education='HS-grad', gender='Female', nativecountry=?)>

In [115]:
##Q4: Write two filter queries

In [116]:
##To filter the number of entries with no 'workclass' value
session.query(Adult).filter(Adult.workclass.like('%?')).count()

2018-08-06 20:26:12,788 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT "Adult_Data".id AS "Adult_Data_id", "Adult_Data".age AS "Adult_Data_age", "Adult_Data".workclass AS "Adult_Data_workclass", "Adult_Data".education AS "Adult_Data_education", "Adult_Data".gender AS "Adult_Data_gender", "Adult_Data".nativecountry AS "Adult_Data_nativecountry" 
FROM "Adult_Data" 
WHERE "Adult_Data".workclass LIKE ?) AS anon_1
2018-08-06 20:26:12,789 INFO sqlalchemy.engine.base.Engine ('%?',)


1836

In [117]:
##To filter data as for the females and order it according to age
fil_female = session.query(Adult).filter_by(gender='Female').order_by('age').all()

2018-08-06 20:27:22,012 INFO sqlalchemy.engine.base.Engine SELECT "Adult_Data".id AS "Adult_Data_id", "Adult_Data".age AS "Adult_Data_age", "Adult_Data".workclass AS "Adult_Data_workclass", "Adult_Data".education AS "Adult_Data_education", "Adult_Data".gender AS "Adult_Data_gender", "Adult_Data".nativecountry AS "Adult_Data_nativecountry" 
FROM "Adult_Data" 
WHERE "Adult_Data".gender = ? ORDER BY "Adult_Data".age
2018-08-06 20:27:22,013 INFO sqlalchemy.engine.base.Engine ('Female',)


In [118]:
fil_female    ##Since the dataset is large, not the entire dataset is visible in the output, will have to scroll down. 
              ##Hence seleced the first lot where we can see the change in order

[<Adult(age='17', workclass='?', education='Undergraduate', gender='Female', nativecountry=United-States)>,
 <Adult(age='17', workclass='Private', education='11th', gender='Female', nativecountry=United-States)>,
 <Adult(age='17', workclass='?', education='11th', gender='Female', nativecountry=United-States)>,
 <Adult(age='17', workclass='Private', education='10th', gender='Female', nativecountry=United-States)>,
 <Adult(age='17', workclass='?', education='11th', gender='Female', nativecountry=United-States)>,
 <Adult(age='17', workclass='Private', education='11th', gender='Female', nativecountry=United-States)>,
 <Adult(age='17', workclass='?', education='11th', gender='Female', nativecountry=United-States)>,
 <Adult(age='17', workclass='Private', education='11th', gender='Female', nativecountry=United-States)>,
 <Adult(age='17', workclass='Private', education='11th', gender='Female', nativecountry=United-States)>,
 <Adult(age='17', workclass='?', education='10th', gender='Female', na

In [119]:
##Q5: Write two function queries

In [120]:
from sqlalchemy import func

In [121]:
#Function 1: To show the number of people in each workclass
session.query(Adult.workclass, func.count(Adult.workclass)).group_by(Adult.workclass).all()

2018-08-06 20:31:47,972 INFO sqlalchemy.engine.base.Engine SELECT "Adult_Data".workclass AS "Adult_Data_workclass", count("Adult_Data".workclass) AS count_1 
FROM "Adult_Data" GROUP BY "Adult_Data".workclass
2018-08-06 20:31:47,973 INFO sqlalchemy.engine.base.Engine ()


[('?', 1836),
 ('Federal-gov', 960),
 ('Local-gov', 2093),
 ('Never-worked', 7),
 ('Private', 22695),
 ('Self-emp-inc', 1116),
 ('Self-emp-not-inc', 2541),
 ('State-gov', 1297),
 ('Without-pay', 14)]

In [123]:
#Function 2:

func_gender = session.query(Adult.gender, func.avg(Adult.age)).group_by(Adult.gender).filter(Adult.nativecountry=='India').all()

2018-08-06 20:34:01,388 INFO sqlalchemy.engine.base.Engine SELECT "Adult_Data".gender AS "Adult_Data_gender", avg("Adult_Data".age) AS avg_1 
FROM "Adult_Data" 
WHERE "Adult_Data".nativecountry = ? GROUP BY "Adult_Data".gender
2018-08-06 20:34:01,389 INFO sqlalchemy.engine.base.Engine ('India',)


In [124]:
func_gender

[('Female', 32.72727272727273), ('Male', 38.766666666666666)]